In [1]:
# unset PYTHONPATH first
from ko_text import *
from ko_crawler import *

In [2]:
nlp = NLP()

# 불러오기

In [3]:
token_df = pd.read_csv('Data/meta_morphs_final.csv', encoding = 'utf-8')

# 용량을 줄이기 위해 '단어 단어' 꼴로 묶어둔 token을 ['단어', '단어'] 꼴로 풀기
token_df['Token'] = [token.split() for token in token_df['Token']]

In [4]:
token_df.head()

,Section,Text,Token,Num of Tokens
0,financial,\n\n\n텀블벅에서 크라우드 펀딩이 이뤄지고 있는 `아침달 시집`.\n\n ...,"[텀블벅, 크라, 우드, 펀딩, 이뤄지고, 아침, 시집, 많지, 않은, 금액, 으로...",263
1,economy,\n\n\n[사진 제공: 연합뉴스]\n\n 유류...,"[유류, 인하, 국제, 유가, 급락, 입어, 국내, 휘발유, 경유, 하락, 특히, ...",166
2,financial,부득이한 사정으로 매월 내는 보험료가 부담이 될 때 계약은 그대로 유지하면서 보험...,"[부득이, 사정, 매월, 내는, 보험료, 부담, 계약, 그대로, 유지, 보험료, 부...",314
3,estate,한때 `미분양의 늪`으로 통하던 경기도 파주시 부동산 시장이 달라지고 있다. 지난해...,"[한때, 미분, 하던, 경기도, 파주시, 부동산, 시장, 달라지고, 분양, 파주, ...",165
4,economy,\n\n\n인디고뱅크의 `미키인서울` 컬래버 맨투맨 <사진제공=월트디즈니코리아>\...,"[인디고, 뱅크, 미키, 서울, 컬래버, 투맨, 월트디즈니, 사의, 마스코트, 미키...",196


In [5]:
token_df.shape

(42331, 4)

# Train Test Split

In [6]:
train_size = round(len(token_df) * 0.8)
np.random.seed(0)
train_index_ls = np.random.choice(token_df.index, train_size, replace = False)
test_index_ls = [x for x in token_df.index if not x in train_index_ls]

In [7]:
train_df = token_df.loc[train_index_ls]
train_df.shape

(33865, 4)

In [8]:
test_df = token_df.loc[test_index_ls]
test_df.shape

(8466, 4)

In [9]:
Counter(train_df['Section'])

Counter({'bio & tech': 1730,
         'business': 5055,
         'culture & art': 4105,
         'economy': 2611,
         'estate': 3958,
         'financial': 763,
         'it': 2125,
         'politics': 3810,
         'society': 3374,
         'stock': 2560,
         'world': 3774})

In [10]:
Counter(test_df['Section'])

Counter({'bio & tech': 430,
         'business': 1324,
         'culture & art': 1045,
         'economy': 638,
         'estate': 979,
         'financial': 184,
         'it': 482,
         'politics': 903,
         'society': 867,
         'stock': 640,
         'world': 974})

# sampling for training classfier

In [11]:
# 한 label마다 학습할 단어의 수
train_batch_size = 1000
test_batch_size = 100

train_token_ls_split, train_tag_ls_split = nlp.oversample_batch(train_df['Token'], train_df['Section'], train_batch_size)
test_token_ls_split, test_tag_ls_split =  nlp.undersample_batch(test_df['Token'],test_df['Section'], test_batch_size)

In [12]:
print(len(train_token_ls_split))
Counter(train_tag_ls_split)

11000


Counter({'bio & tech': 1000,
         'business': 1000,
         'culture & art': 1000,
         'economy': 1000,
         'estate': 1000,
         'financial': 1000,
         'it': 1000,
         'politics': 1000,
         'society': 1000,
         'stock': 1000,
         'world': 1000})

<br>


# 잘 뽑혔는지 확인

In [13]:
' '.join(train_token_ls_split[0])

'SK 텔레콤 새롭게 개편 택시 호출 서비스 티맵 택시 택시 기사 이상 확보 이로써 전국 택시 기사 명의 티맵 택시 호출 있게 됐고 특히 서울시 에선 택시 기사 가입 SK 텔레콤 설명 택시 배차 공률 역시 높아졌다 평균 배차 공률 애플리케이션 개편 전인 수준 이었으나 규모 증가 회사 측은 티맵 택시 호출 수도 애플리케이션 개편 전과 비교 이상 늘어나는 대폭 증가 티맵 택시 탑승객 승객 위치 지인 확인 안심 귀가 라이브 이용 조사 택시 탑승 고객 서비스 이용 택시 위치 도착 예정 시간 이용 택시 정보 본인 희망 가족 지인 보낼 지영 SK 텔레콤 TTS 사업 유닛 티맵 택시 혜택 새로운 기능 고객 택시 기사 분들 빠르게 반응 고객 기사 수요 맞춰 택시 호출 시장 경쟁 바람 일으키고 이용자 혜택 확대 최선'

In [14]:
train_tag_ls_split[0]

'it'

# Doc2Vec Parameter 튜닝

## **set ALPHA as default**

In [15]:
token_ls = train_df['Token'].tolist() + test_df['Token'].tolist()
label_ls = train_df['Section'].tolist() + test_df['Section'].tolist()

In [16]:
len(token_ls), len(label_ls)

(42331, 42331)

In [17]:
%%time

result_dict = {
                'corpus_count' : [],
               'min_count' : [],
               'vector_size' : [],
               'window' : [],
               'n_epochs' : [],
               'accuracy' : [],
               'sample' : [],
               'dm' : [],
              }

testing_section_ls = ['경제','기업','사회','국제','부동산','증권','정치','IT과학','문화']

# 하이퍼 파라미터 튜닝 작업 수행

for dm in [1]:
    for doc2vec_size in ['']:
        if doc2vec_size == '':
            x_split, y_split = token_ls, label_ls

        for sample in [1e-04, 1e-05, 1e-06]:
            for min_count in [1, 5, 15, 50]:
                for vector_size in [100,300]:
                    for window in [5,15]:
                        for n_epochs in [10]:

                            # Doc2Vec 모델 생성
                            nlp.make_Doc2Vec_model(dm = dm,
                                                   min_count = min_count,
                                                   sample = sample,
                                                   vector_size = vector_size,
                                                   window = window,
                                                   dm_mean = 0,
                                                   dm_concat = 0)

                            nlp.build_and_train_Doc2Vec_model(x_split,
                                                              y_split,
                                                              n_epochs = n_epochs)


                            model_name = 'Doc2Vec_dm=%s&cc=%s&vs=%s&win=%s&min=%s&sample=%s&epochs=%s'%(\
                                                                                                                 nlp.Doc2Vec_model.dm,
                                                                                                                   nlp.Doc2Vec_model.corpus_count,
                                                                                                                   nlp.Doc2Vec_model.vector_size,
                                                                                                                   nlp.Doc2Vec_model.window,
                                                                                                                   nlp.Doc2Vec_model.min_count,
                                                                                                                   nlp.Doc2Vec_model.sample,
                                                                                                                   nlp.Doc2Vec_model.epochs)
                            # Doc2Vec 모델 저장
                            nlp.Doc2Vec_model.save('Doc2Vec_model/'+model_name)

                            '''
                            X =nlp.infer_vectors_with_Doc2Vec(train_token_ls_split)

                            from sklearn.decomposition import PCA

                            pca = PCA(n_components=2)
                            X_pca = pca.fit_transform(X)
                            scatter_df = pd.DataFrame(X_pca,
                                                      index = train_tag_ls_split,
                                                      columns = ['x','y'])

                            plt.figure(figsize = (15, 15))

                            for section in testing_section_ls:
                                temp_df = scatter_df[scatter_df.index == section]
                                plt.scatter(temp_df['x'].values, temp_df['y'].values, label = section, c = np.random.rand(3,))

                            plt.legend(loc = 'best')
                            #plt.savefig('Doc2Vec_model/images/'+model_name)
                            #plt.imshow()

                            '''
                            
                            # clf를 각 레이블별 1000개씩 학습, 
                            X_train = nlp.infer_vectors_with_Doc2Vec(train_token_ls_split)
                            y_train = train_tag_ls_split

                            X_test = nlp.infer_vectors_with_Doc2Vec(test_token_ls_split)
                            y_test = test_tag_ls_split


                            clf = LogisticRegression(solver = 'sag',
                                                     multi_class = 'multinomial')


                            clf.fit(X_train, y_train)
                            y_pred = clf.predict(X_test)
                            
                            result_dict['dm'].append(nlp.Doc2Vec_model.dm)
                            result_dict['corpus_count'].append(nlp.Doc2Vec_model.corpus_count)
                            result_dict['min_count'].append(nlp.Doc2Vec_model.min_count)
                            result_dict['vector_size'].append(nlp.Doc2Vec_model.vector_size)
                            result_dict['window'].append(nlp.Doc2Vec_model.window)
                            result_dict['n_epochs'].append(nlp.Doc2Vec_model.epochs)
                            result_dict['sample'].append(nlp.Doc2Vec_model.sample)
                            result_dict['accuracy'].append(accuracy_score(y_pred, y_test))

                            print(accuracy_score(y_pred, y_test))


            pd.DataFrame(result_dict).to_csv('Parameter_tuning_result.csv', index = False)

2018-12-03 00:06:21,310 : INFO : collecting all words and their counts
2018-12-03 00:06:21,311 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-12-03 00:06:21,670 : INFO : PROGRESS: at example #10000, processed 1841400 words (5139084/s), 78283 word types, 11 tags
2018-12-03 00:06:22,040 : INFO : PROGRESS: at example #20000, processed 3711607 words (5068361/s), 107711 word types, 11 tags
2018-12-03 00:06:22,430 : INFO : PROGRESS: at example #30000, processed 5575804 words (4787613/s), 128365 word types, 11 tags
2018-12-03 00:06:22,802 : INFO : PROGRESS: at example #40000, processed 7396756 words (4902799/s), 142020 word types, 11 tags
2018-12-03 00:06:22,905 : INFO : collected 148759 word types and 11 unique tags from a corpus of 42331 examples and 7868234 words
2018-12-03 00:06:22,905 : INFO : Loading a fresh vocabulary
2018-12-03 00:06:23,621 : INFO : effective_min_count=1 retains 148759 unique words (100% of original 148759, drops 0)
2018-12-03 00:

0.8672727272727273


2018-12-03 00:09:21,896 : INFO : PROGRESS: at example #10000, processed 1841400 words (6180924/s), 78283 word types, 11 tags
2018-12-03 00:09:22,206 : INFO : PROGRESS: at example #20000, processed 3711607 words (6036137/s), 107711 word types, 11 tags
2018-12-03 00:09:22,526 : INFO : PROGRESS: at example #30000, processed 5575804 words (5848043/s), 128365 word types, 11 tags
2018-12-03 00:09:22,829 : INFO : PROGRESS: at example #40000, processed 7396756 words (6005082/s), 142020 word types, 11 tags
2018-12-03 00:09:22,920 : INFO : collected 148759 word types and 11 unique tags from a corpus of 42331 examples and 7868234 words
2018-12-03 00:09:22,920 : INFO : Loading a fresh vocabulary
2018-12-03 00:09:23,598 : INFO : effective_min_count=1 retains 148759 unique words (100% of original 148759, drops 0)
2018-12-03 00:09:23,599 : INFO : effective_min_count=1 leaves 7868234 word corpus (100% of original 7868234, drops 0)
2018-12-03 00:09:23,935 : INFO : deleting the raw counts dictionary of 

0.83


2018-12-03 00:12:24,402 : INFO : PROGRESS: at example #10000, processed 1841400 words (6448163/s), 78283 word types, 11 tags
2018-12-03 00:12:24,709 : INFO : PROGRESS: at example #20000, processed 3711607 words (6090537/s), 107711 word types, 11 tags
2018-12-03 00:12:25,018 : INFO : PROGRESS: at example #30000, processed 5575804 words (6054553/s), 128365 word types, 11 tags
2018-12-03 00:12:25,311 : INFO : PROGRESS: at example #40000, processed 7396756 words (6234387/s), 142020 word types, 11 tags
2018-12-03 00:12:25,394 : INFO : collected 148759 word types and 11 unique tags from a corpus of 42331 examples and 7868234 words
2018-12-03 00:12:25,394 : INFO : Loading a fresh vocabulary
2018-12-03 00:12:26,047 : INFO : effective_min_count=1 retains 148759 unique words (100% of original 148759, drops 0)
2018-12-03 00:12:26,048 : INFO : effective_min_count=1 leaves 7868234 word corpus (100% of original 7868234, drops 0)
2018-12-03 00:12:26,376 : INFO : deleting the raw counts dictionary of 

...

0.7463636363636363


2018-12-03 01:59:01,431 : INFO : PROGRESS: at example #10000, processed 1841400 words (5710965/s), 78283 word types, 11 tags
2018-12-03 01:59:01,759 : INFO : PROGRESS: at example #20000, processed 3711607 words (5712022/s), 107711 word types, 11 tags
2018-12-03 01:59:02,136 : INFO : PROGRESS: at example #30000, processed 5575804 words (4951675/s), 128365 word types, 11 tags
2018-12-03 01:59:02,456 : INFO : PROGRESS: at example #40000, processed 7396756 words (5712920/s), 142020 word types, 11 tags
2018-12-03 01:59:02,550 : INFO : collected 148759 word types and 11 unique tags from a corpus of 42331 examples and 7868234 words
2018-12-03 01:59:02,550 : INFO : Loading a fresh vocabulary
2018-12-03 01:59:02,654 : INFO : effective_min_count=50 retains 14516 unique words (9% of original 148759, drops 134243)
2018-12-03 01:59:02,654 : INFO : effective_min_count=50 leaves 7083832 word corpus (90% of original 7868234, drops 784402)
2018-12-03 01:59:02,743 : INFO : deleting the raw counts dictio

0.7463636363636363


2018-12-03 02:00:22,688 : INFO : PROGRESS: at example #10000, processed 1841400 words (5497748/s), 78283 word types, 11 tags
2018-12-03 02:00:23,021 : INFO : PROGRESS: at example #20000, processed 3711607 words (5622946/s), 107711 word types, 11 tags
2018-12-03 02:00:23,355 : INFO : PROGRESS: at example #30000, processed 5575804 words (5597289/s), 128365 word types, 11 tags
2018-12-03 02:00:23,706 : INFO : PROGRESS: at example #40000, processed 7396756 words (5193142/s), 142020 word types, 11 tags
2018-12-03 02:00:23,803 : INFO : collected 148759 word types and 11 unique tags from a corpus of 42331 examples and 7868234 words
2018-12-03 02:00:23,803 : INFO : Loading a fresh vocabulary
2018-12-03 02:00:24,097 : INFO : effective_min_count=50 retains 14516 unique words (9% of original 148759, drops 134243)
2018-12-03 02:00:24,097 : INFO : effective_min_count=50 leaves 7083832 word corpus (90% of original 7868234, drops 784402)
2018-12-03 02:00:24,184 : INFO : deleting the raw counts dictio

0.7509090909090909
CPU times: user 3h 48min 25s, sys: 9min 13s, total: 3h 57min 38s
Wall time: 1h 55min 23s


In [13]:
nlp.Doc2Vec_model.docvecs.vectors_docs.shape

(9, 300)

<br>
